In [1]:
#Imports
from datascience import *
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

https://stockx.com/news/the-2019-data-contest/

The data in this sheet consist of a random sample of all U.S. Off-White x Nike and Yeezy 350 sales from between 9/1/2017 and 2/13/2019. 

To create this sample, StockX took a random, fixed percentage of their sales (X%) for each colorway, on each day, since September 2017. So, for each day the Off-White Jordan 1 was on the market, they randomly selected X% of its sale from each day. (It’s not important to know what X is; all that matters is that it’s a random sample, and that the same fixed X% of sales was selected from every day, for every sneaker).

They've included 8 variables for us to work with: Order Date, Brand, Sneaker Name, Sale Price ($), Retail Price ($), Release Date, Shoe Size, and Buyer State (the U.S. state the buyer shipped to). You can use whatever variables you want in the analysis; you can use 1 variable, or you can use all 8. And remember, every row in the spreadsheet represents an individual StockX sale. There are no averages or order counts; this is just a random sample of daily sales data.

In [31]:
sneakerdata = pd.read_excel('StockX-Data-Contest-2019-3.xlsx')

In [3]:
sneakerdata.shape

(99956, 8)

In [4]:
sneakerdata.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island


##### Data Cleaning

Let's check for any missing data.

In [5]:
sneakerdata.isnull().sum()

Order Date      0
Brand           0
Sneaker Name    0
Sale Price      0
Retail Price    0
Release Date    0
Shoe Size       0
Buyer Region    0
dtype: int64

There are no missing data. Let's now check if the data types of the columns are consistent with the values in the columns.

In [6]:
sneakerdata.dtypes

Order Date      datetime64[ns]
Brand                   object
Sneaker Name            object
Sale Price             float64
Retail Price             int64
Release Date    datetime64[ns]
Shoe Size              float64
Buyer Region            object
dtype: object

The data is consistent with the type of values.

The data seems to be about shoe sales, but there is no profit column. Profit is the difference between the sale price and the retail price. Let's add a profit column to our data frame.

In [32]:
sneakerdata['Profit'] = sneakerdata['Sale Price'] - sneakerdata['Retail Price']

In [33]:
sneakerdata.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Profit
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,877.0
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,465.0
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California,470.0
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky,855.0
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island,608.0


Let's re-order the columns for aesthetics so that 'Profit' is next to 'Sale Price' and 'Retail Price'

In [34]:
sneakerdata = sneakerdata[['Order Date', 'Brand', 'Sneaker Name', 'Sale Price', 'Retail Price','Profit','Release Date','Shoe Size','Buyer Region']]
sneakerdata.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Profit,Release Date,Shoe Size,Buyer Region
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,877.0,2016-09-24,11.0,California
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,465.0,2016-11-23,11.0,California
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,470.0,2016-11-23,11.0,California
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,855.0,2016-11-23,11.5,Kentucky
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,608.0,2017-02-11,11.0,Rhode Island


Let's take a look at the different shoe brands.

In [35]:
sneakerdata['Brand'].unique()

array([' Yeezy', 'Off-White'], dtype=object)

There seems to be an uncessary space before the word Yeezy in the data. Let us get rid of it using RegEx. Using this idea: https://stackoverflow.com/questions/25698710/replace-all-occurrences-of-a-string-in-a-pandas-dataframe-python  we will replace ' Yeezy' with 'Yeezy'

In [36]:
sneakerdata[sneakerdata['Brand'] == " Yeezy"].shape[0] ##checking initial rows

72162

In [38]:
sneakerdata[sneakerdata['Brand'] == "Off-White"].shape[0] ##checking initial rows

27794

In [39]:
sneakerdata['Brand'] = sneakerdata['Brand'].replace({' Yeezy': 'Yeezy'}, regex=True)

In [41]:
sneakerdata[sneakerdata['Brand'] == "Yeezy"].shape[0] ##rows match, the regex worked

72162

In [42]:
sneakerdata[sneakerdata['Brand'] == "Off-White"].shape[0] ##rows match, the regex worked

27794

##### Data Exploration

In my data exploration, I will try to answer some thought-provoking questions about the dataset that I will come up with myself in order to find some interesting results.

#### 1. Which shoe size generated the most profit? For Yeezy's? For Off-whites?

First, let us see the unique shoe sizes for yeezy's and off-whites, respectively.

In [55]:
yeezy = sneakerdata[sneakerdata['Brand'] == "Yeezy"]

In [56]:
offwhite = sneakerdata[sneakerdata['Brand'] == "Off-White"]

In [65]:
yeezy.groupby(['Shoe Size']).mean()['Profit']

Shoe Size
3.5     129.000000
4.0     107.985032
4.5     114.205781
5.0     123.892903
5.5     127.116979
6.0     140.270868
6.5     143.208804
7.0     125.619676
7.5     126.265436
8.0     138.008518
8.5     141.041237
9.0     132.979267
9.5     144.985291
10.0    143.860640
10.5    143.796080
11.0    152.163142
11.5    142.121689
12.0    141.474104
12.5    168.353464
13.0    151.477338
13.5    172.523810
14.0    156.344124
14.5    230.440476
16.0    482.066667
17.0    615.333333
Name: Profit, dtype: float64

In [66]:
offwhite.groupby(['Shoe Size']).mean()['Profit']

Shoe Size
3.5      305.333333
4.0      407.490314
4.5      416.850825
5.0      438.117647
5.5      403.327526
6.0      428.859061
6.5      405.141375
7.0      444.071095
7.5      489.797904
8.0      525.135565
8.5      492.487328
9.0      535.184513
9.5      511.911618
10.0     530.122029
10.5     462.443742
11.0     504.027723
11.5     458.529817
12.0     491.154280
12.5     536.092308
13.0     472.214069
14.0     551.646847
15.0     490.515385
16.0    1490.250000
17.0    2060.000000
Name: Profit, dtype: float64

From these results based on the average profit for each size, smaller sizes tend to do better than bigger sizes.

#### 2. Which region tends to buy more Yeezy's? More Off-whites?

In [74]:
yeezy.groupby(['Buyer Region']).count().reset_index().sort_values('Order Date', ascending = False).set_index('Buyer Region')['Order Date'][0:10]

Buyer Region
California       13113
New York         12103
Oregon            5396
Florida           4484
Texas             4455
New Jersey        3371
Illinois          2732
Pennsylvania      2396
Michigan          2209
Massachusetts     2189
Name: Order Date, dtype: int64

In [75]:
offwhite.groupby(['Buyer Region']).count().reset_index().sort_values('Order Date', ascending = False).set_index('Buyer Region')['Order Date'][0:10]

Buyer Region
California       6236
New York         4422
Oregon           2285
Florida          1892
Texas            1421
New Jersey       1349
Illinois         1050
Massachusetts     714
Pennsylvania      713
Virginia          605
Name: Order Date, dtype: int64

Based on these results, it seems that the top 10 regions that tend to buy one brand tend  to buy the other, with California being at the top.

#### 3. 

Some ideas: Hypothesis test maybe for profit on the different type of brand?

P.S. Add my datascienceportfoliio tab to my github website.